In [8]:
## Assignment 1 Intelligent Systems 

In [9]:
import pygad
import pandas as pd
import numpy as np

# Load the JSON file into a pandas DataFrame


In [10]:
#  Converting the preference data into an appropriate format 


df = pd.read_json('easy_dataset_1.json')
#First drop unnecessary data and the them into variables 
num_papers = df.loc[0,"num_papers"]
df = df.drop(columns="num_papers")
num_reviewers = df.loc[0,"num_reviewers"]
df = df.drop(columns="num_reviewers")
reviewer_capacity = df.loc[0,"reviewer_capacity"]
df = df.drop(columns ="reviewer_capacity")
min_reviews_per_paper = df.loc[0,"min_reviews_per_paper"]
df = df.drop(columns="min_reviews_per_paper")
max_reviews_per_paper = df.loc[0,"max_reviews_per_paper"]
df = df.drop(columns="max_reviews_per_paper")
df = df.drop(columns="dataset_id")
print(df)




preferences_matrix = np.array(df["preferences"].tolist())
friendships_matrix = np.array(df["friendships"].tolist())
authorship_matrix = np.array(df["authorship"].tolist())
print(num_papers)

       preferences      friendships       authorship
0  [3, 2, 1, 5, 3]  [0, 0, 0, 0, 0]  [0, 0, 0, 0, 0]
1  [5, 2, 4, 2, 2]  [0, 0, 1, 0, 0]  [0, 1, 0, 0, 0]
2  [4, 3, 1, 2, 4]  [0, 1, 0, 0, 0]  [0, 0, 0, 0, 0]
3  [4, 2, 2, 1, 1]  [0, 0, 0, 0, 0]  [1, 1, 0, 0, 1]
4  [4, 3, 4, 1, 5]  [0, 0, 0, 0, 0]  [0, 0, 1, 1, 0]
5


In [11]:
# Example assign matrix for given dataset 
assign_matrixnp = np.zeros((5,5), dtype= int)
assign_matrixnp[0,2] = 1
assign_matrixnp[0,3] = 1
assign_matrixnp[0,4] = 1
assign_matrixnp[0,1] = 1
assign_matrixnp[1,2] = 1
assign_matrixnp[2,4] = 1
assign_matrixnp[3,0] = 1
assign_matrixnp[4,3] = 1
print(assign_matrixnp)

[[0 1 1 1 1]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [1 0 0 0 0]
 [0 0 0 1 0]]


In [103]:
#compute fitness value in absolute or relative 
def f(assign_matrix, preferences_matrix, friendships_matrix, authorship_matrix): 
    fit_val = 0 
   
    # penalty scores table/distribution
    preference_points = np.array([[0,1,2,3,4,5],[3,2,1,0,-1,-2]])
    #penalty_reviewer_cap = 3
    #penalty_authorship = 7
    #penalty_friendship = 5

    #penalty_low_reviewcount= 5 
    #penalty_high_reviewcount = 1 

    ### higher penalties to see if crossover helps 

    penalty_reviewer_cap = 5
    penalty_authorship = 9
    penalty_friendship = 7

    penalty_low_reviewcount= 7
    penalty_high_reviewcount = 3 
    

    # iterate over reviewers and papers
    for i in range(len(assign_matrix[:,0])): 
        for j in range(len(assign_matrix[0,:])): 
            # add the preference penalty for the assigned paper
            if assign_matrix[i,j] == 1: 
                index = preferences_matrix[i,j]
                fit_val += preference_points[1, index]
                # get other y of reviewed paper j 
                for y in range(len(assign_matrix[0,:])): 
                    if authorship_matrix[y,j] == 1:
                        author = y
                #if reviewer and author are friends assign friendconstraint penalty 
                if friendships_matrix[author,i] == 1: 
                    fit_val+= penalty_friendship
                    print("friendship penalty")
            # if assigned reviewer is also author add authorconstraint penalty
            if assign_matrix[i,j] == 1 and authorship_matrix[i,j] ==1:
                fit_val+= penalty_authorship     
                print("authorship penalty")       
        
    # penalties for reviewer constraints violations              
    for t in range(len(assign_matrix[:,0])): 
        x = np.sum(assign_matrix[i,t])
        if x > reviewer_capacity:
            fit_val += penalty_reviewer_cap
            print("penalty reviewer cap")
    
    # penalties with the amount of reviews needed is not met worse for less reviews than for 2 much reviews
    for z in range(len(assign_matrix[0,:])):
        x = np.sum(assign_matrix[:,z])
        if x < min_reviews_per_paper:
            fit_val += (min_reviews_per_paper - x)*penalty_low_reviewcount
            print("penalty for to less reviews for paper")
        if x > max_reviews_per_paper:
            fit_val += (x - max_reviews_per_paper)*penalty_high_reviewcount
    # problem that low fitness function is better -> assign minus      
    fit_val = -1*fit_val    
    return fit_val 

print(f(assign_matrixnp, preferences_matrix, friendships_matrix, authorship_matrix ))



authorship penalty
authorship penalty
penalty for to less reviews for paper
penalty for to less reviews for paper
penalty for to less reviews for paper
penalty for to less reviews for paper
penalty for to less reviews for paper
-67


In [113]:
#Utilize PyGad 

#pyGad takes as a solution max 2D Arrays to flatten reviewer x paper array x num solutions in num solutions x values
def fitness_function(ga_instance, solution, solution_idx):
    return f(solution.reshape(num_reviewers,num_papers), preferences_matrix, friendships_matrix, authorship_matrix)   

#create initial position with random values  
def custom_initialpop(num_solutions, num_reviewers, num_papers):
    pop = np.random.randint(0,2, size=(num_solutions, num_reviewers, num_papers))
    return pop.reshape((num_solutions,-1))


#initiate ga_instance
ga_instance = pygad.GA(fitness_func=fitness_function,
                       crossover_type=custom_crossover,
                       num_generations=10,      # number of generations
                       num_parents_mating=2,    #                   
                       sol_per_pop=10,          # solutions per pop
                       initial_population = custom_initialpop(10,num_reviewers,num_papers), #initialpopulation
                       gene_type=int,                          
                       num_genes= num_reviewers*num_papers,
                       gene_space = [0,1])       
                       
                                                     
#ga_instance.run()
aa = custom_initialpop(10,5,5)
#print(aa.reshape(10,num_reviewers,num_papers))

#print(ga_instance.cal_pop_fitness())

### Example code for parents to work with 
num_parents = 4  
num_genes = 25   
# this is just random not constraints respected 
parents = np.random.randint(0, 2, size=(num_parents, num_genes))

#maybe also generate parents that respect constraints for testing 



## custom crossover functions that return offspring which fullfill the constraints 

def custom_crossover(parents, offspring_size, ga_instance):
    offspring = []

    ## Do initial crossover inherit each column/ paper assigned to authors for either parent 1 or 2 

    for i in range(parents.shape[0]):
        parent1 = parents[i % parents.shape[0], :].copy()
        parent2 = parents[(i+1) % parents.shape[0], :].copy()

        # SWAP Papers (Columns) the Assignmentmatrix

        # reshape for easier computation 
        parent1 = parent1.reshape(num_reviewers,num_papers)
        parent2 = parent2.reshape(num_reviewers,num_papers)
        
        # equally the amount of columns given to the child by each parent
        
        hereditary_parent1 = num_papers // 2 #whole number divide operator 
        hereditary_parent2 = num_papers - hereditary_parent1
        # calcs a 1D-Array with indices which columns(papers) are given to the child afterwards
        indices_parent1 = np.random.choice(range(num_papers), size=hereditary_parent1, replace=False)
        indices_parent2 = [i for i in range(num_papers) if i not in indices_parent1]

        # computes child out of the randomized indices
        new_child = np.zeros_like(parent1)
        new_child[:, indices_parent1] = parent1[:, indices_parent1]
        new_child[:, indices_parent2] = parent2[:, indices_parent2]
        
        offspring.append(new_child)

    offspring = np.array(offspring)
    

    ## Check if reviewer capacity is violated and that remove papers 

    # for every child
    for i in range(offspring.shape[0]):
        # for every row / reviewer 
        for j in range(offspring.shape[1]): 
            if np.sum(offspring[i,j,:]) > reviewer_capacity: 
                # Number of assignments that need to be removed from this reviewer
                to_remove = np.sum(offspring[i,j,:]) - reviewer_capacity
                # perform the remove operations as many as to remove 
                for y in range(to_remove): 
                    #get assigned indexes and choose a random idx from that and remove the assignment
                    idx_assigned = np.where(offspring[i,j,:] == 1)[0]
                    idx_delete = np.random.choice(idx_assigned, size=1, replace=False)
                    offspring[i,j,idx_delete] = 0
                    
    ## add reviewer assignment to fullfill review_requirements just add assignment if no other constraints are violated

    # iterate over every child in offspring
    for i in range(offspring.shape[0]):
        #iterate over every col / paper 
        for j in range(offspring.shape[2]):
            
            # do corrections if review requierements now dont fit anymore
            if np.sum(offspring[i,:,j]) < min_reviews_per_paper: 
                
                # available reviewers dont have the paper assigned yet and are under their reviewer_capacity
                available_reviewers = np.where((offspring[i,:,j] == 0) & (np.sum(offspring[i,:,:], axis=1) < reviewer_capacity))[0]
                
                for reviewer in available_reviewers: 
                    #check whether potential reviewers is friends or author 
                    is_friend = False
                    is_author = False
                    if authorship_matrix[reviewer, j] == 1: 
                        is_friend = True
                    authors = np.where(authorship_matrix[:,j] == 1)[0]
                    
                    for author in authors: 
                        if friendships_matrix[author, reviewer] == 1: 
                            is_author = True
                    if (is_friend == False & is_author == False): 
                        offspring[i,reviewer,j] = 1
                        break
    return offspring                    


     
x = custom_crossover(parents, (4,25), ga_instance)


p1 = f(parents[0].reshape(5,5), preferences_matrix, friendships_matrix, authorship_matrix)
p2 = f(parents[1].reshape(5,5), preferences_matrix, friendships_matrix, authorship_matrix)  
p3 = f(parents[2].reshape(5,5), preferences_matrix, friendships_matrix, authorship_matrix)
p4 = f(parents[3].reshape(5,5), preferences_matrix, friendships_matrix, authorship_matrix)

print((p1+p2+p3+p4)/2)

x0 = f(x[0], preferences_matrix, friendships_matrix, authorship_matrix)
x1 = f(x[1], preferences_matrix, friendships_matrix, authorship_matrix)
x2 = f(x[2], preferences_matrix, friendships_matrix, authorship_matrix)
x3 = f(x[3], preferences_matrix, friendships_matrix, authorship_matrix)

print("initial mean:",(p1+p2+p3+p4)/2)

print("crossover mean:",(x1+x2+x3+x0)/2)





penalty for to less reviews for paper
penalty for to less reviews for paper
penalty for to less reviews for paper
penalty for to less reviews for paper
penalty for to less reviews for paper
authorship penalty
authorship penalty
penalty for to less reviews for paper
authorship penalty
authorship penalty
authorship penalty
authorship penalty
penalty for to less reviews for paper
penalty for to less reviews for paper
authorship penalty
authorship penalty
authorship penalty
penalty for to less reviews for paper
penalty for to less reviews for paper
penalty for to less reviews for paper
-109.0
authorship penalty
authorship penalty
penalty for to less reviews for paper
penalty for to less reviews for paper
authorship penalty
authorship penalty
penalty for to less reviews for paper
penalty for to less reviews for paper
authorship penalty
authorship penalty
penalty for to less reviews for paper
penalty for to less reviews for paper
penalty for to less reviews for paper
authorship penalty
penal

In [ ]:
#initiate ga_instance
ga_instance = pygad.GA(fitness_func=fitness_function,
                       num_generations=10,      # number of generations
                       num_parents_mating=2,    #                   
                       sol_per_pop=10,          # solutions per pop
                       initial_population = custom_initialpop(10,num_reviewers,num_papers), #initialpopulation
                       gene_type=int,                          
                       num_genes= num_reviewers*num_papers,
                       gene_space = [0,1])       
                       
                                                     
#ga_instance.run()
aa = custom_initialpop(10,5,5)
print(aa.reshape(10,num_reviewers,num_papers))

print(ga_instance.cal_pop_fitness())

[[[0 1 1 1 1]
  [0 0 0 0 0]
  [1 0 0 0 1]
  [1 0 0 0 0]
  [1 1 0 0 1]]

 [[0 0 1 0 1]
  [0 0 1 1 0]
  [0 1 0 1 1]
  [0 1 1 1 1]
  [1 1 1 1 1]]

 [[1 1 1 0 0]
  [0 1 0 0 0]
  [0 1 1 1 0]
  [1 0 1 0 0]
  [1 0 0 1 1]]

 [[0 0 1 1 0]
  [0 0 1 0 1]
  [0 1 1 0 0]
  [1 1 1 0 0]
  [1 1 0 0 1]]

 [[0 0 0 0 0]
  [0 0 1 1 0]
  [0 0 0 1 0]
  [0 1 1 1 0]
  [1 1 0 0 1]]

 [[1 1 0 1 0]
  [1 1 1 1 0]
  [0 1 0 1 0]
  [0 1 0 0 0]
  [0 1 1 1 0]]

 [[1 1 0 1 0]
  [1 1 1 0 1]
  [0 0 1 0 1]
  [1 1 0 0 1]
  [0 1 0 1 1]]

 [[0 0 0 1 0]
  [1 1 0 0 1]
  [0 0 0 0 0]
  [0 1 1 1 1]
  [1 1 0 1 0]]

 [[0 1 1 0 0]
  [0 0 0 1 0]
  [1 1 1 1 0]
  [0 1 1 0 1]
  [1 0 0 0 1]]

 [[0 1 1 1 1]
  [1 0 1 1 1]
  [1 0 0 0 0]
  [1 1 0 0 1]
  [1 0 0 0 1]]]
[-39 -37 -36 -48 -59 -48 -27 -16 -34 -45]


c:\Users\noah_\miniconda3\envs\intelsys\lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


In [ ]:
# TRASH 
assign_matrix = pd.DataFrame(0, index=[f"Reviewer {i+1}" for i in range(5)], 
                                columns=[f"Paper {j+1}" for j in range(5)])